In [1]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
import random

In [2]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama3")

In [3]:
transformer=LLMGraphTransformer(llm=llm)

In [4]:
# Replace these with your Neo4j credentials and URI
NEO4J_URI = "YOUR_URI"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "YOUR_PASSWORD"


In [5]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session() as session:
            result = session.run(query, parameters)
            return list(result)  # Fetch all records and return as a list
        
    def create_node(self, label, properties):
        labels_str = ":".join(label)
        query = f"CREATE (n:{labels_str} $properties)"
        return self.query(query, parameters={"properties": properties})
    
    def create_relationship(self, start_node_id, end_node_id, rel_type, properties=None):
        query = (
            "MATCH (start), (end) "
            "WHERE start.id = $start_node_id AND end.id = $end_node_id "
            f"CREATE (start)-[r:{rel_type} $properties]->(end)"
        )
        parameters = {
            "start_node_id": start_node_id,
            "end_node_id": end_node_id,
            "properties": properties or {}
        }
        return self.query(query, parameters)

In [6]:
class Neo4jGraph:
    def __init__(self, connection):
        self.connection = connection
    
    def add_graph_documents(self, graph_documents, base_entity_label=True, include_source=True):
        for graph_document in graph_documents:
            # Process nodes
            for node in graph_document.nodes:
                labels = [node.type]
                if base_entity_label:
                    labels.append("BaseEntity")
                properties = {"id": node.id}
                self.connection.create_node(labels, properties)
            
            # Process relationships
            for relationship in graph_document.relationships:
                properties = {}  # Add properties if needed
                self.connection.create_relationship(relationship.source.id, relationship.target.id, relationship.type, properties)


In [7]:
conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
graph = Neo4jGraph(conn)


In [8]:

# Example text
text = """
Vivekananda was born as Narendranath Datta (name shortened to Narendra or Naren)in a Bengali Kayastha family in his ancestral home at 3 Gourmohan Mukherjee Street in Calcutta, the capital of British India, on 12 January 1863 during the Makar Sankranti festival. He belonged to a traditional family and was one of nine siblings.His father, Vishwanath Datta, was an attorney at the Calcutta High Court. Durgacharan Datta, Narendra's grandfather was a Sanskrit and Persian scholar who left his family and became a monk at age twenty-five. His mother, Bhubaneswari Devi, was a devout housewife.The progressive, rational attitude of Narendra's father and the religious temperament of his mother helped shape his thinking and personality. Narendranath was interested in spirituality from a young age and used to meditate before the images of deities such as Shiva, Rama, Sita, and Mahavir Hanuman.He was fascinated by wandering ascetics and monks. Narendra was mischievous and restless as a child, and his parents often had difficulty controlling him. His mother said, "I prayed to Shiva for a son and he has sent me one of his demons".

"""


# Create a document from the text
document = Document(page_content=text)

# Convert document to graph documents
graph_documents = transformer.convert_to_graph_documents([document])

# Extract nodes and relationships from the graph documents
nodes = graph_documents[0].nodes
relationships = graph_documents[0].relationships


# Print nodes and relationships
print(f"Nodes: {nodes}")
print(f"Relationships: {relationships}")

Nodes: [Node(id='Durgacharan Datta', type='Person'), Node(id='Narendranath Datta', type='Person'), Node(id='Bhubaneswari Devi', type='Person'), Node(id="Vivekananda's mother", type='Person'), Node(id='Shiva', type='Deity'), Node(id='Vivekananda', type='Person'), Node(id='Vishwanath Datta', type='Person')]
Relationships: [Relationship(source=Node(id='Vivekananda', type='Person'), target=Node(id='Narendranath Datta', type='Person'), type='BORN_AS'), Relationship(source=Node(id='Vivekananda', type='Person'), target=Node(id='Vishwanath Datta', type='Person'), type='CHILD_OF'), Relationship(source=Node(id='Vivekananda', type='Person'), target=Node(id='Bhubaneswari Devi', type='Person'), type='CHILD_OF'), Relationship(source=Node(id='Narendranath Datta', type='Person'), target=Node(id='Vivekananda', type='Person'), type='FATHER_OF'), Relationship(source=Node(id='Durgacharan Datta', type='Person'), target=Node(id='Narendranath Datta', type='Person'), type='FATHER_OF'), Relationship(source=Nod

In [9]:
graph_documents

[GraphDocument(nodes=[Node(id='Durgacharan Datta', type='Person'), Node(id='Narendranath Datta', type='Person'), Node(id='Bhubaneswari Devi', type='Person'), Node(id="Vivekananda's mother", type='Person'), Node(id='Shiva', type='Deity'), Node(id='Vivekananda', type='Person'), Node(id='Vishwanath Datta', type='Person')], relationships=[Relationship(source=Node(id='Vivekananda', type='Person'), target=Node(id='Narendranath Datta', type='Person'), type='BORN_AS'), Relationship(source=Node(id='Vivekananda', type='Person'), target=Node(id='Vishwanath Datta', type='Person'), type='CHILD_OF'), Relationship(source=Node(id='Vivekananda', type='Person'), target=Node(id='Bhubaneswari Devi', type='Person'), type='CHILD_OF'), Relationship(source=Node(id='Narendranath Datta', type='Person'), target=Node(id='Vivekananda', type='Person'), type='FATHER_OF'), Relationship(source=Node(id='Durgacharan Datta', type='Person'), target=Node(id='Narendranath Datta', type='Person'), type='FATHER_OF'), Relations

In [10]:
# Add graph documents to Neo4j
graph.add_graph_documents(graph_documents)

# Close connection
conn.close()

print("Nodes and relationships added successfully to Neo4j using Neo4jGraph.")

Nodes and relationships added successfully to Neo4j using Neo4jGraph.
